In [ ]:
# This project reads in a webpage containing a dataset directory with datasets in format of images, when the user gives a random
# image from the whole dataset, it should return the name of the directory where the picture is from 

In [ ]:
# Main method, calling functions in order

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/CSE_163_Project"

/content/drive/.shortcut-targets-by-id/1ziAjH650mc891BLsYvUXN1BqvL0La2KX/CSE 163 Project


In [33]:
#open("/content/drive/MyDrive/CSE_163_Project/archive")

IsADirectoryError: ignored

# 新段落

In [34]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from google.colab import drive

class LoadData(Dataset):
    def __init__(self, data_dir):
        self.__data_dir = data_dir
        self.__categories = [os.path.join(self.__data_dir, name) for name in os.listdir(self.__data_dir) if os.path.isdir(os.path.join(self.__data_dir, name))]
        self.__label_list = []
        self.__image_list = []
        
        for category in self.__categories:
            image_paths = [os.path.join(category, name) for name in os.listdir(category) if name.endswith('.jpg') or name.endswith('.jpeg') or name.endswith('.png')]
            label = os.path.basename(category)

            for file_path in image_paths:
                img = Image.open(file_path)
                img = img.resize((228, 228))
                img_array = np.array(img)
                img_array = img_array / 255.0

                self.__label_list.append(label)
                self.__image_list.append(img_array)
        
        self.__labels = sorted(list(set(self.__label_list)))
        self.__label_to_idx = {label: i for i, label in enumerate(self.__labels)}
        self.__idx_to_label = {i: label for i, label in enumerate(self.__labels)}

    def __getitem__(self, index):
        label = self.__label_list[index]
        image = self.__image_list[index]
        label_idx = self.__label_to_idx[label]
        label_tensor = torch.tensor(label_idx, dtype=torch.long)
        image_tensor = torch.tensor(image, dtype=torch.float)
        return image_tensor, label_tensor

    def __len__(self):
        return len(self.__image_list)

    def get_labels(self):
        return self.__labels

    def get_label(self, index):
        return self.__idx_to_label[index]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
# import matplotlib as plt

# Define the neural network architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 258)  # Output size set to 256 for 256 object categories

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        # print(x.numpy().shape) to match
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5) # reshape the output of the layer from a 4D (batch size, num_filters, height, width)
                                   # to a 2D (batch_size, flatterned_feature_size) where flattened_feature_size = num_filters
                                   # * height * width
                                   # Finished the flattening since this 2D tensor has rows for input sample and column
                                   # for feature
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load the dataset
transform = transforms.Compose(
    [transforms.Resize((32, 32)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


# Split train and test set and their loaders
trainset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/CSE_163_Project/archive', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/CSE_163_Project/archive', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)
# create a list of class names from the file names in the directory
classes = []
for name in os.listdir('/content/drive/MyDrive/CSE_163_Project/archive'):
  if len(name.split('.')) == 2:
    classes.append(name.split('.')[1])


# Define the loss function and optimizer
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Train the network
for epoch in range(2): # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): # get the inputs in a list of [inputs, labels]
        inputs, labels = data
        optimizer.zero_grad() # zero the parameter gradients
        outputs = net(inputs) # forward + backward + optimize
        #print(outputs.detach())
        #print(labels)
        #target = np.zeros((4, 257))
        #target[labels,: ] = 1
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

# Save the trained model
PATH = '/content/drive/MyDrive/CSE_163_Project'
torch.save(net.state_dict(), PATH)







# Test the network on the test data
dataiter = iter(testloader)
images, labels = next(dataiter)
# Image Printing for 5 target labels in first 5 cycles of collection
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))


# Evaluation of the Model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')


# Overall Evaluation of Model Performance Classified by Labels
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1



# show the distribution in plots, mean accuracies amongest all accuracies



            
# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')



In [ ]:
# Split the whole dataset into training and test sets

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
# import matplotlib as plt

# Define the neural network architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 256)  # Output size set to 256 for 256 object categories

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        # print(x.numpy().shape) to match
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5) # reshape the output of the layer from a 4D (batch size, num_filters, height, width)
                                   # to a 2D (batch_size, flatterned_feature_size) where flattened_feature_size = num_filters
                                   # * height * width
                                   # Finished the flattening since this 2D tensor has rows for input sample and column
                                   # for feature
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F



# Load the dataset
transform = transforms.Compose(
    [transforms.Resize((32, 32)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# Test the network on the test data
testset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/CSE_163_Project/archive', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)
dataiter = iter(testloader)
images, labels = next(dataiter)
classes = []
for name in os.listdir('/content/drive/MyDrive/CSE_163_Project/archive'):
  if len(name.split('.')) == 2:
    classes.append(name.split('.')[1])
# Image Printing for 5 target labels in first 5 cycles of collection
print('GroundTruth: ', ' '.join('%5s' % classes[j] for j in range(5)))
net = CNN()

# Evaluation of the Model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

GroundTruth:  DS_Store ostrich mailbox hot-air-balloon buddha-101
tensor([ 33, 253,  78,  33])
tensor([33, 78, 78, 33])
tensor([ 33,  78,  33, 253])
tensor([ 78,  33,  33, 190])
tensor([78, 33, 33, 33])
tensor([ 33, 190,  33,  33])
tensor([197,  78,  33,  33])
tensor([ 78, 190, 190,  33])
tensor([78, 78, 33, 33])
tensor([ 33,  33, 190,  33])
tensor([ 33,  78, 190,  33])
tensor([33, 33, 78, 33])
tensor([190,  33,  33,  33])
tensor([33, 78, 33, 78])
tensor([ 33,  33,  78, 197])
tensor([78, 33, 33, 33])
tensor([197,  33,  33,  33])
tensor([ 33, 190,  78,  33])
tensor([33, 33, 33, 33])
tensor([190,  33, 190,  33])
tensor([33, 33, 33, 33])
tensor([190,  33,  33,  33])
tensor([190,  33,  33,  33])
tensor([190, 197,  33,  33])
tensor([78, 78, 33, 78])
tensor([78, 33, 33, 33])
tensor([197,  33,  33,  33])
tensor([ 33,  33, 197,  33])
tensor([ 33,  33,  33, 190])
tensor([33, 33, 33, 78])
tensor([33, 33, 33, 33])
tensor([78, 33, 33, 33])
tensor([33, 33, 33, 33])
tensor([ 33, 197, 197,  78])
tens

KeyboardInterrupt: ignored